In [ ]:
# Collect desired calendar properties from initial input
# 1- access data source
# 2- creat calendar structure for each calendar
# 3- Fill structure with calendar properties and calendar data
# 4- Process calendar data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# specify the file path, you might need to adjust this
file_path = "TST00-Baseline.xlsx"
# Load the calendar data into a DataFrame
calendar_df = pd.read_excel(file_path, sheet_name='CALENDAR')
# Load a sheet into a dataframe by its name
project_df = pd.read_excel(file_path, sheet_name='PROJECT')

In [ ]:
calendar_df

In [ ]:
calendar_struct = calendar_df.iloc[0].to_dict()
# get first key
first_key = next(iter(calendar_struct))

# delete first key-value pair
calendar_struct.pop(first_key)

In [ ]:
calendar_struct

In [ ]:
# Assuming 'clndr_data' is a column in 'calendar_df'
clndr_data_split = calendar_df['clndr_data'].str.split('\\(0\\|\\|', expand=True).stack()

# Create new dataframe with each output of split operation as a new row
cal_data_df = pd.DataFrame(clndr_data_split, columns=['Data'])

# Reset the index
cal_data_df = cal_data_df.reset_index(drop=True)

In [ ]:
cal_data_df

In [ ]:
start_index = cal_data_df[cal_data_df['Data'].str.contains('DaysOfWeek')].index[0]
end_index = cal_data_df[cal_data_df['Data'].str.contains('VIEW')].index[0]

# Create a new DataFrame by selecting rows between "DaysOfWeek" and "VIEW"
DaysOfWeek_df = cal_data_df.loc[start_index + 1: end_index - 1]

In [ ]:
# Initialize a list to hold the data
WeekDays_data = []

# Iterate over rows in DaysOfWeek_df
for i in range(len(DaysOfWeek_df['Data'])):

    if DaysOfWeek_df['Data'].iloc[i][0] != '0':
        # Add new entry and mark it as the current entry
        current_entry = [DaysOfWeek_df['Data'].iloc[i][0], '0', '0']
        WeekDays_data.append(current_entry)

    elif 's' in DaysOfWeek_df['Data'].iloc[i] and 'f' in DaysOfWeek_df['Data'].iloc[i]:
        # Modify 'StartingHour' and 'EndingHour' of current entry
        start_s = DaysOfWeek_df['Data'].iloc[i].index('s') + 2
        start_f = DaysOfWeek_df['Data'].iloc[i].index('f') + 2
        current_entry[1] = DaysOfWeek_df['Data'].iloc[i][start_s: start_s + 5]
        current_entry[2] = DaysOfWeek_df['Data'].iloc[i][start_f: start_f + 5]

# Create DataFrame
WeekDays = pd.DataFrame(WeekDays_data, columns=['DayNumber', 'StartingHour', 'EndingHour'])

In [ ]:
WeekDays

In [ ]:
# Get index of row containing 'Exceptions' in 'Data' column
exceptions_index = cal_data_df[cal_data_df['Data'].str.contains('Exceptions')].index[0]

# Select all rows after that index
exceptions_df = cal_data_df.iloc[exceptions_index + 1:]

In [ ]:
# Initialize a list to hold the data
Exceptions_data = []

# Iterate over rows in exceptions_df
for i in range(len(exceptions_df['Data'])):

    if 'd' in exceptions_df['Data'].iloc[i]:
        # Add new entry and mark it as the current entry
        idx = exceptions_df['Data'].iloc[i].index('d')
        current_entry = [exceptions_df['Data'].iloc[i][idx + 2:idx + 7], '0', '0']
        Exceptions_data.append(current_entry)

        # If next row contains 's' and 'f', modify 'StartingHour' and 'EndingHour' of current entry
        if i < len(exceptions_df['Data']) - 1 and 's' in exceptions_df['Data'].iloc[i + 1] and 'f' in \
                exceptions_df['Data'].iloc[i + 1]:
            start_s = exceptions_df['Data'].iloc[i + 1].index('s') + 2
            start_f = exceptions_df['Data'].iloc[i + 1].index('f') + 2
            current_entry[1] = exceptions_df['Data'].iloc[i + 1][start_s: start_s + 5]
            current_entry[2] = exceptions_df['Data'].iloc[i + 1][start_f: start_f + 5]

# Create DataFrame
Exceptions = pd.DataFrame(Exceptions_data, columns=['DayNumber', 'StartingHour', 'EndingHour'])

In [ ]:
Exceptions

In [ ]:
project_df

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# This function converts Excel's date serial number to python datetime
def excel_date_to_dt(date):
    return (pd.to_datetime('1900-01-01') + pd.to_timedelta(date - 2, unit='d')).date()

# Find the maximum DayNumber in the Exceptions dataframe 
max_daynumber = Exceptions['DayNumber'].max()

# Convert max_daynumber to integer if it's not already
max_daynumber = int(max_daynumber)

# Convert the max day number to a date
max_daynumber_date = excel_date_to_dt(max_daynumber)

# Get the first scd_end_date from the project_df dataframe
scd_end_date = project_df['scd_end_date'].iloc[0]

# Make sure scd_end_date is in a date format
scd_end_date = pd.to_datetime(scd_end_date).date()

# Compare the date from max_daynumber_date and scd_end_date, take the later one
if max_daynumber_date > scd_end_date:
    project_finish_date = max_daynumber_date
else:
    project_finish_date = scd_end_date

In [ ]:
project_start_date = pd.to_datetime(project_df['plan_start_date']).iloc[0]
#project_finish_date = pd.to_datetime(project_df['scd_end_date']).iloc[0]

In [ ]:
project_start_date

In [ ]:
project_finish_date

In [ ]:
start_date = project_start_date.date()

In [ ]:
start_date

In [ ]:
WeekDays

In [ ]:
# Reindexing DayNumber in WeekDays to match Python's date convention Monday=0, Sunday=6
mapping = {1:6, 2:0, 3:1, 4:2, 5:3, 6:4, 7:5}
WeekDays['DayNumber'] = WeekDays['DayNumber'].astype(int)
WeekDays['DayNumber'] = WeekDays['DayNumber'].map(mapping)

In [ ]:
WeekDays

In [ ]:
# creating date range
Calendar_df = pd.DataFrame({'Date': pd.date_range(start=project_start_date, end=project_finish_date)})

# formatting date to dd-mm-yyyy format
Calendar_df['Date'] = Calendar_df['Date'].dt.strftime('%d-%m-%Y')

# getting the weekday number where Monday=0, Sunday=6
Calendar_df['DayNumber'] = pd.to_datetime(Calendar_df['Date'], format='%d-%m-%Y').dt.dayofweek

# merging Calendar_df with WeekDays on 'DayNumber' to get 'StartingHour' and 'EndingHour'
Calendar_df = pd.merge(Calendar_df, WeekDays, on='DayNumber', how='left')

# removing the 'DayNumber' column as it's no longer needed
Calendar_df = Calendar_df.drop(columns='DayNumber')

In [ ]:
Calendar_df

In [ ]:
print(Calendar_df.dtypes)

In [ ]:
# Fixing the "0" in hours column
Calendar_df['StartingHour'] = Calendar_df['StartingHour'].replace({"0": "00:00"})
Calendar_df['EndingHour'] = Calendar_df['EndingHour'].replace({"0": "00:00"})

# Converting columns to desired datatypes
Calendar_df['Date'] = pd.to_datetime(Calendar_df['Date'], dayfirst=True)
Calendar_df['StartingHour'] = pd.to_datetime(Calendar_df['StartingHour'], format='%H:%M').dt.time
Calendar_df['EndingHour'] = pd.to_datetime(Calendar_df['EndingHour'], format='%H:%M').dt.time

In [ ]:
Calendar_df

In [ ]:
import pandas as pd
from datetime import datetime

# Converting 'StartingHour' and 'EndingHour' to AM/PM string format
Calendar_df['StartingHour'] = Calendar_df['StartingHour'].apply(lambda x: x.strftime('%I:%M %p'))
Calendar_df['EndingHour'] = Calendar_df['EndingHour'].apply(lambda x: x.strftime('%I:%M %p'))

# Create temporary datetime objects for 'StartingHour' and 'EndingHour' to calculate 'WorkingHours'
Calendar_df['TempStart'] = pd.to_datetime(Calendar_df['StartingHour'], format='%I:%M %p')
Calendar_df['TempEnd'] = pd.to_datetime(Calendar_df['EndingHour'], format='%I:%M %p')

# Calculate WorkingHours and convert to total hours as numeric
Calendar_df['WorkingHours'] = ((Calendar_df['TempEnd'] - Calendar_df['TempStart']).dt.seconds) / 3600

# Drop the temporary columns
Calendar_df = Calendar_df.drop(columns=['TempStart', 'TempEnd'])

In [ ]:
Calendar_df

In [ ]:
import numpy as np

Calendar_df['Type'] = np.where(Calendar_df['WorkingHours'] == 0, 'Non work', 'Standard')

In [ ]:
Calendar_df

In [ ]:
def excel_date_to_dt(date):
    return (pd.to_datetime('1900-01-01') + pd.to_timedelta(date - 2, unit='d')).date()

# Ensure 'DayNumber' column is numeric
Exceptions['DayNumber'] = pd.to_numeric(Exceptions['DayNumber'])

# Apply the function to 'DayNumber' column
Exceptions['Date'] = Exceptions['DayNumber'].apply(excel_date_to_dt)

# Convert 'Date' in Exceptions dataframe to datetime
Exceptions['Date'] = pd.to_datetime(Exceptions['Date'])

In [ ]:
Exceptions

In [ ]:
# Merge Calendar_df and Exceptions_df on Date:
df = pd.merge(Calendar_df, Exceptions, on='Date', how='left')

# Function to apply modifications
def modify_rows(row):
    if row['StartingHour_y'] == 0 and row['EndingHour_y'] == 0:
        row['StartingHour_x'] = '12:00 AM'
        row['EndingHour_x'] = '12:00 AM'
        row['WorkingHours'] = 0
        row['Type'] = 'Non work'
    return row

# Apply modifications
df = df.apply(modify_rows, axis=1)

# Drop unnecessary columns and rename columns
df = df.drop(columns=['StartingHour_y', 'EndingHour_y', 'DayNumber'])
df = df.rename(columns={'StartingHour_x':'StartingHour',
                        'EndingHour_x':'EndingHour'})

# The DataFrame 'df' now holds the data with all the required modifications

In [ ]:
df

In [ ]:
# Replace '0' with '00:00' and assign it back to the DataFrame
Exceptions['StartingHour'] = Exceptions['StartingHour'].replace('0', '00:00')
Exceptions['EndingHour'] = Exceptions['EndingHour'].replace('0', '00:00')

In [ ]:
Exceptions

In [ ]:
# Convert 'StartingHour' and 'EndingHour' columns to datetime
Exceptions['StartingHour'] = pd.to_datetime(Exceptions['StartingHour'], format='%H:%M').dt.strftime('%I:%M %p')
Exceptions['EndingHour'] = pd.to_datetime(Exceptions['EndingHour'], format='%H:%M').dt.strftime('%I:%M %p')

In [ ]:
Exceptions

In [ ]:
# Create temporary DataFrames which parses the time columns to datetime
temp_start = pd.to_datetime(Exceptions['StartingHour'].astype(str), format='%I:%M %p')
temp_end = pd.to_datetime(Exceptions['EndingHour'].astype(str), format='%I:%M %p')

# Perform the subtraction Operation and convert it to seconds, then divide by 3600 to get hours
Exceptions['WorkingHours'] = (temp_end - temp_start).dt.total_seconds() / 3600

In [ ]:
Exceptions

In [ ]:
Exceptions_zero_hours = Exceptions[Exceptions['WorkingHours'] == 0]
for idx, row in Exceptions_zero_hours.iterrows():
    date = row['Date']
    if date in Calendar_df['Date'].values:
        Calendar_df.loc[Calendar_df['Date'] == date, 'StartingHour'] = row['StartingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'EndingHour'] = row['EndingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'WorkingHours'] = row['WorkingHours']
        Calendar_df.loc[Calendar_df['Date'] == date, 'Type'] = "Non work"

In [ ]:
Calendar_df

In [ ]:
# Filter rows in 'Exceptions' where 'WorkingHours' is not equal to 0
exceptions_nonzero_hours = Exceptions[Exceptions['WorkingHours'] != 0]

# Find corresponding rows in 'Calendar_df'
for index, row in exceptions_nonzero_hours.iterrows():
    date = row['Date']

    # If 'Calendar_df' contains the date
    if date in Calendar_df['Date'].values:
        # Update corresponding 'StartingHour', 'EndingHour', 'WorkingHours and 'Type' in 'Calendar_df'
        Calendar_df.loc[Calendar_df['Date'] == date, 'StartingHour'] = row['StartingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'EndingHour'] = row['EndingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'WorkingHours'] = row['WorkingHours']
        Calendar_df.loc[Calendar_df['Date'] == date, 'Type'] = "Exception"

In [ ]:
Calendar_df

In [ ]:
# Assuming Calendar_df is your DataFrame
calendar_file_path = 'Calendar.xlsx'  # you may want to include full path here
with pd.ExcelWriter(calendar_file_path, engine='openpyxl') as writer:
    Calendar_df.to_excel(writer, sheet_name='Calendar', index=False)